In [1]:
!pip install deepgram-sdk

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 111 kB 40.8 MB/s 
     |████████████████████████████████| 271 kB 61.5 MB/s 
     |████████████████████████████████| 160 kB 67.9 MB/s 
     |████████████████████████████████| 144 kB 76.1 MB/s 


In [5]:
%%writefile deepgram_test.py
from deepgram import Deepgram
import asyncio, json

# The API key you created in step 1
DEEPGRAM_API_KEY = 'd65c64d60bd88adb0b65ec52770bc069e96723c5'

AUDIO_URL = 'https://static.deepgram.com/examples/Bueller-Life-moves-pretty-fast.wav'

async def main():
    # Initializes the Deepgram SDK
    dg_client = Deepgram(DEEPGRAM_API_KEY)
    source = {'url': AUDIO_URL}

    # print('Requesting transcript...')
    # print('Your file may take up to a couple minutes to process.')
    # print('While you wait, did you know that Deepgram accepts over 40 audio file formats? Even MP4s.')
    # print('To learn more about customizing your transcripts check out developers.deepgram.com.')

    response = await dg_client.transcription.prerecorded(source,  {'punctuate': True})
    print(json.dumps(response, indent=4))

asyncio.run(main())

Overwriting deepgram_test.py


In [6]:
!python deepgram_test.py > text.json

In [16]:
import re
import json
import math

file = open('text.json', 'r')
res = []
for lne in file:
  # print(lne)e
  lst = re.findall('"transcript": "(.*)"', lne)
  if(len(lst)>0):
    print(lst[0])
    break
# print(res)

Yep. I said it before, and I'll say it again. Life moves pretty fast. You don't stop and look around once in a while. You could miss it.


# Real time audio (no need)

In [20]:
%%writefile deepgram_test_streaming.py
from deepgram import Deepgram
import asyncio, json

# The API key you created in step 1
DEEPGRAM_API_KEY = 'd65c64d60bd88adb0b65ec52770bc069e96723c5'

# Name and extension of the file you downloaded (e.g. sample.wav)
PATH_TO_FILE = 'Bueller-Life-moves-pretty-fast.wav'

async def main():
    # Initializes the Deepgram SDK
    dg_client = Deepgram(DEEPGRAM_API_KEY)

    # Creates a websocket connection to Deepgram
    socket = await dg_client.transcription.live({'punctuate': True})

    # print('Connection Opened!')

    # Handle sending audio to the socket
    async def process_audio(connection):
        # Grab your audio file
        with open(PATH_TO_FILE, 'rb') as audio:
            # Chunk up the audio to send
            CHUNK_SIZE_BYTES = 8192
            CHUNK_RATE_SEC = 0.001
            chunk = audio.read(CHUNK_SIZE_BYTES)
            while chunk:
                connection.send(chunk)
                await asyncio.sleep(CHUNK_RATE_SEC)
                chunk = audio.read(CHUNK_SIZE_BYTES)

            # Indicate that we've finished sending data
            await connection.finish()

    # Receive transcriptions based on sent streams and write them to the console
    socket.register_handler(socket.event.CLOSE, lambda _: print('Connection closed.'))

    # Print incoming transcription objects
    socket.register_handler(socket.event.TRANSCRIPT_RECEIVED, print)

    # Send the audio to the socket
    await process_audio(socket)

asyncio.run(main())

Overwriting deepgram_test_streaming.py


In [19]:
!python deepgram_test_streaming.py > text_streaming.json